# This is for tuning the hyperparameter

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Load pickled data
import pickle
import matplotlib.pyplot as plt

# TODO: Fill this in based on where you saved the training and testing data

training_file = "test.p"
validation_file="valid.p"
testing_file = "train.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']

X_test, y_test = test['features'], test['labels']

In [8]:
import numpy as np
print('train='+str(X_train.shape))
print('test='+str(X_test.shape))
print('validate='+str(X_valid.shape))
# print(X_train.shape)
train_shape=X_train.shape
test_shape=X_test.shape
valid_shape=X_valid.shape
labels_number=len(np.unique(y_train))
labels_number

train=(12630, 32, 32, 3)
test=(34799, 32, 32, 3)
validate=(4410, 32, 32, 3)


43

# Here i am normalising the images by diving it by 255.0 and making it between 0 and 1.i,e Minmax Scaler.
### This is my decision based on expericece and literature to use  Normalisation (and that too Min-Max) in CNN in place on Standarization.
    However if your find that Standarzation will work better in this case 
    then kindly let me know once. 

In [9]:
##Normalisation
X_train=X_train/255.0
X_test=X_test/255.0
X_valid=X_valid/255.0


In [10]:
import tensorflow as tf
from tensorflow import estimator,metrics,sigmoid,acos
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPool2D,Dropout,Flatten,Conv2D
from tensorflow.keras.activations import relu

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# here i am building an function for tuning the hyperparameters, with (32,32,3) inputs and 43 outputs.

In [11]:
def build_models(hp):
    model=Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filter',min_value=32,max_value=96,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(32,32,3)
        ), 
        Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
               kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
               activation='relu'
        ),
        MaxPool2D(pool_size=(2,2)),
        Dropout(hp.Float('d1',min_value=.1,max_value=.5)),

        Conv2D(filters=hp.Int('conv_3_filter',min_value=32,max_value=96,step=16),
               kernel_size=hp.Choice('conv_3_kernel',values=[3,5]),
               activation='relu'
        ),
        MaxPool2D(pool_size=(2,2)),
        Dropout(hp.Float('d2',min_value=.1,max_value=.5)),

        Conv2D(filters=hp.Int('conv_4_filter',min_value=32,max_value=128,step=16),kernel_size=(3,3),padding="same", activation="relu"),
#         MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Dropout(hp.Float('d3',min_value=.1,max_value=.5)),
      
        Conv2D(filters=hp.Int('conv_5_filter',min_value=32,max_value=256,step=16),kernel_size=(3,3),padding="same", activation="relu"),
#         MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Dropout(hp.Float('d4',min_value=.1,max_value=.5)),

        Conv2D(filters=hp.Int('conv_6_filter',min_value=32,max_value=128,step=16),kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Dropout(hp.Float('d5',min_value=.1,max_value=.5)),

        Dense(units=hp.Int('dense_1_units',
              min_value=30,max_value=130,step=10),
              activation='relu'
        ),
        Dropout(hp.Float('d4',min_value=0.1,max_value=.5)),

        
        Flatten(),
        
        Dense(units=hp.Int('dense_2_units',
              min_value=128,max_value=512,step=32),
              activation='relu'
        ),
        
        Dropout(hp.Float('d5',min_value=0.1,max_value=.5)),

        Dense(43,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-1,1e-3])),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# importing library for search of appropiate values

In [77]:
from kerastuner import RandomSearch,HyperParameters #importing HyperParamter and RandomSearch

In [84]:
#now using random serch for this model
tuner_search=RandomSearch(build_models,objective='val_accuracy',max_trials=15,directory='output',project_name='p6')

# Here i am using random search instance for search best parameters out there, this may take an hour or half depending on the GPU
### Please donot run this next line until necessary it may take a hour to complete on

In [85]:
#seaching hyper parameters for this using this module i.e modle will do backpropogation,frontpropogation and other 
# stuffs to carry on this model and find best parameter 
tuner_search.search(X_train,y_train,epochs=15,validation_data=(X_valid,y_valid))

Train on 12630 samples, validate on 4410 samples
Epoch 1/15
12630/12630 [==============================] - ETA: 5:34 - loss: 3.7646 - accuracy: 0.03 - ETA: 1:11 - loss: 57613602.3793 - accuracy: 0.043 - ETA: 41s - loss: 32007558.5586 - accuracy: 0.052 - ETA: 30s - loss: 22159080.5585 - accuracy: 0.05 - ETA: 24s - loss: 16945180.0860 - accuracy: 0.05 - ETA: 20s - loss: 13717527.4227 - accuracy: 0.05 - ETA: 17s - loss: 11522723.6227 - accuracy: 0.05 - ETA: 16s - loss: 9933382.9272 - accuracy: 0.0517 - ETA: 14s - loss: 8729344.2332 - accuracy: 0.047 - ETA: 13s - loss: 7785631.7319 - accuracy: 0.046 - ETA: 12s - loss: 7026058.2479 - accuracy: 0.045 - ETA: 11s - loss: 6401520.6114 - accuracy: 0.049 - ETA: 11s - loss: 5878956.2114 - accuracy: 0.052 - ETA: 10s - loss: 5435261.6686 - accuracy: 0.053 - ETA: 10s - loss: 5053840.0397 - accuracy: 0.052 - ETA: 9s - loss: 4722440.9211 - accuracy: 0.052 - ETA: 9s - loss: 4431896.0313 - accuracy: 0.05 - ETA: 8s - loss: 4174974.7252 - accuracy: 0.05 - 

12630/12630 [==============================] - ETA: 6s - loss: 3.4741 - accuracy: 0.15 - ETA: 5s - loss: 3.5203 - accuracy: 0.05 - ETA: 5s - loss: 3.5318 - accuracy: 0.04 - ETA: 5s - loss: 3.4833 - accuracy: 0.05 - ETA: 5s - loss: 3.4653 - accuracy: 0.05 - ETA: 5s - loss: 3.4877 - accuracy: 0.04 - ETA: 5s - loss: 3.4958 - accuracy: 0.04 - ETA: 5s - loss: 3.4906 - accuracy: 0.05 - ETA: 5s - loss: 3.4865 - accuracy: 0.05 - ETA: 5s - loss: 3.4841 - accuracy: 0.05 - ETA: 5s - loss: 58.0667 - accuracy: 0.057 - ETA: 5s - loss: 230.5311 - accuracy: 0.05 - ETA: 5s - loss: 211.6087 - accuracy: 0.06 - ETA: 5s - loss: 195.6042 - accuracy: 0.05 - ETA: 5s - loss: 181.8825 - accuracy: 0.05 - ETA: 5s - loss: 169.9927 - accuracy: 0.05 - ETA: 5s - loss: 159.5901 - accuracy: 0.05 - ETA: 4s - loss: 150.4065 - accuracy: 0.05 - ETA: 4s - loss: 142.2480 - accuracy: 0.05 - ETA: 4s - loss: 134.9454 - accuracy: 0.05 - ETA: 4s - loss: 128.3709 - accuracy: 0.05 - ETA: 4s - loss: 122.4235 - accuracy: 0.05 - ETA: 

12630/12630 [==============================] - ETA: 6s - loss: 3.7146 - accuracy: 0.0000e+ - ETA: 5s - loss: 3.5706 - accuracy: 0.0375   - ETA: 5s - loss: 3.5328 - accuracy: 0.05 - ETA: 5s - loss: 3.5369 - accuracy: 0.05 - ETA: 5s - loss: 3.5256 - accuracy: 0.04 - ETA: 5s - loss: 3.5085 - accuracy: 0.05 - ETA: 5s - loss: 3.5107 - accuracy: 0.04 - ETA: 5s - loss: 3.5089 - accuracy: 0.05 - ETA: 5s - loss: 3.5122 - accuracy: 0.05 - ETA: 5s - loss: 3.4972 - accuracy: 0.04 - ETA: 5s - loss: 3.4932 - accuracy: 0.04 - ETA: 5s - loss: 3.4928 - accuracy: 0.04 - ETA: 5s - loss: 3.4872 - accuracy: 0.04 - ETA: 5s - loss: 3.4923 - accuracy: 0.04 - ETA: 5s - loss: 3.4949 - accuracy: 0.05 - ETA: 5s - loss: 3.4979 - accuracy: 0.05 - ETA: 4s - loss: 3.4924 - accuracy: 0.05 - ETA: 4s - loss: 3.4928 - accuracy: 0.05 - ETA: 4s - loss: 3.4960 - accuracy: 0.04 - ETA: 4s - loss: 3.4970 - accuracy: 0.04 - ETA: 4s - loss: 3.4987 - accuracy: 0.04 - ETA: 4s - loss: 3.4972 - accuracy: 0.05 - ETA: 4s - loss: 3.495

12630/12630 [==============================] - ETA: 6s - loss: 3.4589 - accuracy: 0.06 - ETA: 5s - loss: 3.5065 - accuracy: 0.04 - ETA: 5s - loss: 3.5317 - accuracy: 0.04 - ETA: 5s - loss: 3.4814 - accuracy: 0.04 - ETA: 5s - loss: 3.4564 - accuracy: 0.04 - ETA: 5s - loss: 3.4396 - accuracy: 0.04 - ETA: 5s - loss: 3.4567 - accuracy: 0.04 - ETA: 5s - loss: 3.4524 - accuracy: 0.05 - ETA: 5s - loss: 3.4680 - accuracy: 0.05 - ETA: 5s - loss: 3.4712 - accuracy: 0.04 - ETA: 5s - loss: 3.4702 - accuracy: 0.04 - ETA: 5s - loss: 3.4836 - accuracy: 0.04 - ETA: 5s - loss: 3.4928 - accuracy: 0.04 - ETA: 5s - loss: 3.4900 - accuracy: 0.04 - ETA: 5s - loss: 3.4989 - accuracy: 0.04 - ETA: 5s - loss: 3.5001 - accuracy: 0.04 - ETA: 4s - loss: 3.5017 - accuracy: 0.04 - ETA: 4s - loss: 3.5078 - accuracy: 0.04 - ETA: 4s - loss: 3.5061 - accuracy: 0.04 - ETA: 4s - loss: 3.5101 - accuracy: 0.04 - ETA: 4s - loss: 3.5105 - accuracy: 0.04 - ETA: 4s - loss: 3.5108 - accuracy: 0.04 - ETA: 4s - loss: 3.5123 - accu

12630/12630 [==============================] - ETA: 6s - loss: 3.4918 - accuracy: 0.03 - ETA: 5s - loss: 3.5015 - accuracy: 0.03 - ETA: 5s - loss: 3.4914 - accuracy: 0.05 - ETA: 5s - loss: 3.4944 - accuracy: 0.06 - ETA: 5s - loss: 3.4658 - accuracy: 0.06 - ETA: 5s - loss: 3.4712 - accuracy: 0.06 - ETA: 5s - loss: 3.4704 - accuracy: 0.06 - ETA: 5s - loss: 3.4678 - accuracy: 0.06 - ETA: 5s - loss: 3.4740 - accuracy: 0.05 - ETA: 5s - loss: 3.4691 - accuracy: 0.06 - ETA: 5s - loss: 3.4813 - accuracy: 0.05 - ETA: 5s - loss: 3.4749 - accuracy: 0.05 - ETA: 5s - loss: 3.4825 - accuracy: 0.05 - ETA: 5s - loss: 3.4814 - accuracy: 0.05 - ETA: 5s - loss: 3.4818 - accuracy: 0.05 - ETA: 5s - loss: 3.4828 - accuracy: 0.05 - ETA: 4s - loss: 3.4863 - accuracy: 0.05 - ETA: 4s - loss: 3.4833 - accuracy: 0.06 - ETA: 4s - loss: 3.4894 - accuracy: 0.06 - ETA: 4s - loss: 3.4931 - accuracy: 0.06 - ETA: 4s - loss: 3.4927 - accuracy: 0.06 - ETA: 4s - loss: 3.4899 - accuracy: 0.05 - ETA: 4s - loss: 3.4880 - accu

...

11392/12630 [==========================>...] - ETA: 13s - loss: 3.5814 - accuracy: 0.125 - ETA: 13s - loss: 3.5465 - accuracy: 0.052 - ETA: 13s - loss: 3.5339 - accuracy: 0.056 - ETA: 13s - loss: 3.5010 - accuracy: 0.049 - ETA: 12s - loss: 3.4823 - accuracy: 0.062 - ETA: 13s - loss: 3.4646 - accuracy: 0.065 - ETA: 13s - loss: 3.4942 - accuracy: 0.060 - ETA: 12s - loss: 3.5026 - accuracy: 0.056 - ETA: 12s - loss: 3.5084 - accuracy: 0.058 - ETA: 12s - loss: 3.5024 - accuracy: 0.057 - ETA: 12s - loss: 3.4852 - accuracy: 0.059 - ETA: 12s - loss: 3.4859 - accuracy: 0.055 - ETA: 12s - loss: 3.4946 - accuracy: 0.056 - ETA: 12s - loss: 3.4959 - accuracy: 0.059 - ETA: 12s - loss: 3.5041 - accuracy: 0.056 - ETA: 12s - loss: 3.5018 - accuracy: 0.055 - ETA: 12s - loss: 3.4971 - accuracy: 0.054 - ETA: 12s - loss: 3.4966 - accuracy: 0.058 - ETA: 12s - loss: 3.4938 - accuracy: 0.056 - ETA: 12s - loss: 3.4920 - accuracy: 0.056 - ETA: 12s - loss: 3.4862 - accuracy: 0.057 - ETA: 12s - loss: 3.4839 - acc

12630/12630 [==============================] - ETA: 1s - loss: 3.4893 - accuracy: 0.05 - ETA: 1s - loss: 3.4894 - accuracy: 0.05 - ETA: 1s - loss: 3.4890 - accuracy: 0.05 - ETA: 1s - loss: 3.4885 - accuracy: 0.05 - ETA: 1s - loss: 3.4886 - accuracy: 0.05 - ETA: 1s - loss: 3.4883 - accuracy: 0.05 - ETA: 0s - loss: 3.4889 - accuracy: 0.05 - ETA: 0s - loss: 3.4887 - accuracy: 0.05 - ETA: 0s - loss: 3.4887 - accuracy: 0.05 - ETA: 0s - loss: 3.4887 - accuracy: 0.05 - ETA: 0s - loss: 3.4891 - accuracy: 0.05 - ETA: 0s - loss: 3.4896 - accuracy: 0.05 - ETA: 0s - loss: 3.4894 - accuracy: 0.05 - ETA: 0s - loss: 3.4891 - accuracy: 0.05 - ETA: 0s - loss: 3.4888 - accuracy: 0.05 - ETA: 0s - loss: 3.4892 - accuracy: 0.05 - ETA: 0s - loss: 3.4889 - accuracy: 0.05 - ETA: 0s - loss: 3.4890 - accuracy: 0.05 - ETA: 0s - loss: 3.4884 - accuracy: 0.05 - ETA: 0s - loss: 3.4880 - accuracy: 0.05 - ETA: 0s - loss: 3.4873 - accuracy: 0.05 - 16s 1ms/sample - loss: 3.4869 - accuracy: 0.0565 - val_loss: 3.5885 - v

11488/12630 [==========================>...] - ETA: 16s - loss: 3.6022 - accuracy: 0.093 - ETA: 16s - loss: 3.4279 - accuracy: 0.083 - ETA: 18s - loss: 3.4458 - accuracy: 0.070 - ETA: 17s - loss: 3.3914 - accuracy: 0.057 - ETA: 17s - loss: 3.4200 - accuracy: 0.053 - ETA: 17s - loss: 3.4308 - accuracy: 0.050 - ETA: 18s - loss: 3.4415 - accuracy: 0.055 - ETA: 18s - loss: 3.4773 - accuracy: 0.050 - ETA: 18s - loss: 3.4942 - accuracy: 0.054 - ETA: 18s - loss: 3.4837 - accuracy: 0.050 - ETA: 17s - loss: 3.4799 - accuracy: 0.047 - ETA: 17s - loss: 3.4877 - accuracy: 0.051 - ETA: 17s - loss: 3.4956 - accuracy: 0.050 - ETA: 16s - loss: 3.4982 - accuracy: 0.053 - ETA: 16s - loss: 3.4894 - accuracy: 0.056 - ETA: 16s - loss: 3.4761 - accuracy: 0.062 - ETA: 16s - loss: 3.4822 - accuracy: 0.060 - ETA: 15s - loss: 3.4681 - accuracy: 0.062 - ETA: 15s - loss: 3.4755 - accuracy: 0.059 - ETA: 15s - loss: 3.4783 - accuracy: 0.059 - ETA: 14s - loss: 3.4752 - accuracy: 0.061 - ETA: 14s - loss: 3.4732 - acc

12630/12630 [==============================] - ETA: 1s - loss: 3.4874 - accuracy: 0.05 - ETA: 1s - loss: 3.4868 - accuracy: 0.05 - ETA: 1s - loss: 3.4862 - accuracy: 0.05 - ETA: 0s - loss: 3.4860 - accuracy: 0.05 - ETA: 0s - loss: 3.4854 - accuracy: 0.05 - ETA: 0s - loss: 3.4856 - accuracy: 0.05 - ETA: 0s - loss: 3.4853 - accuracy: 0.05 - ETA: 0s - loss: 3.4849 - accuracy: 0.05 - ETA: 0s - loss: 3.4852 - accuracy: 0.05 - ETA: 0s - loss: 3.4853 - accuracy: 0.05 - ETA: 0s - loss: 3.4852 - accuracy: 0.05 - ETA: 0s - loss: 3.4852 - accuracy: 0.05 - ETA: 0s - loss: 3.4853 - accuracy: 0.05 - ETA: 0s - loss: 3.4852 - accuracy: 0.05 - ETA: 0s - loss: 3.4846 - accuracy: 0.05 - ETA: 0s - loss: 3.4846 - accuracy: 0.05 - ETA: 0s - loss: 3.4848 - accuracy: 0.05 - ETA: 0s - loss: 3.4847 - accuracy: 0.05 - ETA: 0s - loss: 3.4849 - accuracy: 0.05 - ETA: 0s - loss: 3.4847 - accuracy: 0.05 - ETA: 0s - loss: 3.4844 - accuracy: 0.05 - ETA: 0s - loss: 3.4844 - accuracy: 0.05 - ETA: 0s - loss: 3.4842 - accu

INFO:tensorflow:Oracle triggered exit


In [86]:
model=tuner_search.get_best_models(num_models=1)[0]#using to get best model out of this

# yes here only, i have now used model.predict_classes to value of y_predicted and that is stored in an matrices a

In [90]:
from sklearn.metrics import accuracy_score

# fitting the model again 

In [107]:
history=model.fit(X_train,y_train,epochs=10,validation_data=(X_valid, y_valid))

Train on 12630 samples, validate on 4410 samples
Epoch 1/10
12630/12630 [==============================] - ETA: 3:22 - loss: 0.2162 - accuracy: 0.96 - ETA: 2:03 - loss: 0.1866 - accuracy: 0.96 - ETA: 1:40 - loss: 0.1330 - accuracy: 0.97 - ETA: 1:24 - loss: 0.1060 - accuracy: 0.98 - ETA: 1:28 - loss: 0.0966 - accuracy: 0.98 - ETA: 1:09 - loss: 0.0936 - accuracy: 0.98 - ETA: 50s - loss: 0.1361 - accuracy: 0.9625 - ETA: 43s - loss: 0.1493 - accuracy: 0.959 - ETA: 41s - loss: 0.1397 - accuracy: 0.962 - ETA: 46s - loss: 0.1485 - accuracy: 0.960 - ETA: 39s - loss: 0.1347 - accuracy: 0.965 - ETA: 46s - loss: 0.1255 - accuracy: 0.968 - ETA: 43s - loss: 0.1238 - accuracy: 0.968 - ETA: 40s - loss: 0.1237 - accuracy: 0.968 - ETA: 41s - loss: 0.1224 - accuracy: 0.967 - ETA: 37s - loss: 0.1266 - accuracy: 0.965 - ETA: 38s - loss: 0.1233 - accuracy: 0.965 - ETA: 36s - loss: 0.1271 - accuracy: 0.965 - ETA: 36s - loss: 0.1256 - accuracy: 0.966 - ETA: 36s - loss: 0.1281 - accuracy: 0.966 - ETA: 43s - l

12630/12630 [==============================] - ETA: 9s - loss: 0.0345 - accuracy: 1.00 - ETA: 10s - loss: 0.2027 - accuracy: 0.979 - ETA: 7s - loss: 0.1297 - accuracy: 0.982 - ETA: 7s - loss: 0.1339 - accuracy: 0.97 - ETA: 7s - loss: 0.1213 - accuracy: 0.97 - ETA: 7s - loss: 0.1081 - accuracy: 0.98 - ETA: 6s - loss: 0.1028 - accuracy: 0.97 - ETA: 6s - loss: 0.0930 - accuracy: 0.97 - ETA: 6s - loss: 0.0862 - accuracy: 0.97 - ETA: 6s - loss: 0.0803 - accuracy: 0.97 - ETA: 6s - loss: 0.0772 - accuracy: 0.97 - ETA: 6s - loss: 0.0779 - accuracy: 0.97 - ETA: 6s - loss: 0.0750 - accuracy: 0.98 - ETA: 6s - loss: 0.0743 - accuracy: 0.98 - ETA: 6s - loss: 0.0735 - accuracy: 0.97 - ETA: 6s - loss: 0.0765 - accuracy: 0.97 - ETA: 6s - loss: 0.0759 - accuracy: 0.97 - ETA: 6s - loss: 0.0739 - accuracy: 0.97 - ETA: 6s - loss: 0.0721 - accuracy: 0.97 - ETA: 6s - loss: 0.0700 - accuracy: 0.97 - ETA: 6s - loss: 0.0685 - accuracy: 0.97 - ETA: 6s - loss: 0.0674 - accuracy: 0.97 - ETA: 5s - loss: 0.0705 - a

12630/12630 [==============================] - ETA: 6s - loss: 0.0351 - accuracy: 0.96 - ETA: 5s - loss: 0.0693 - accuracy: 0.98 - ETA: 5s - loss: 0.0784 - accuracy: 0.97 - ETA: 5s - loss: 0.0815 - accuracy: 0.97 - ETA: 5s - loss: 0.0681 - accuracy: 0.97 - ETA: 5s - loss: 0.0597 - accuracy: 0.97 - ETA: 5s - loss: 0.0625 - accuracy: 0.97 - ETA: 5s - loss: 0.0676 - accuracy: 0.97 - ETA: 5s - loss: 0.0739 - accuracy: 0.97 - ETA: 5s - loss: 0.0795 - accuracy: 0.97 - ETA: 5s - loss: 0.0838 - accuracy: 0.97 - ETA: 5s - loss: 0.0822 - accuracy: 0.97 - ETA: 5s - loss: 0.0801 - accuracy: 0.97 - ETA: 5s - loss: 0.0801 - accuracy: 0.97 - ETA: 5s - loss: 0.0801 - accuracy: 0.97 - ETA: 4s - loss: 0.0789 - accuracy: 0.97 - ETA: 4s - loss: 0.0764 - accuracy: 0.97 - ETA: 4s - loss: 0.0779 - accuracy: 0.97 - ETA: 4s - loss: 0.0796 - accuracy: 0.97 - ETA: 4s - loss: 0.0793 - accuracy: 0.97 - ETA: 4s - loss: 0.0805 - accuracy: 0.97 - ETA: 4s - loss: 0.0831 - accuracy: 0.97 - ETA: 4s - loss: 0.0841 - accu

12630/12630 [==============================] - ETA: 6s - loss: 0.2292 - accuracy: 0.90 - ETA: 5s - loss: 0.0888 - accuracy: 0.96 - ETA: 5s - loss: 0.0543 - accuracy: 0.98 - ETA: 5s - loss: 0.0430 - accuracy: 0.98 - ETA: 5s - loss: 0.0360 - accuracy: 0.98 - ETA: 5s - loss: 0.0726 - accuracy: 0.98 - ETA: 5s - loss: 0.0810 - accuracy: 0.98 - ETA: 5s - loss: 0.0791 - accuracy: 0.98 - ETA: 5s - loss: 0.0898 - accuracy: 0.97 - ETA: 5s - loss: 0.0886 - accuracy: 0.97 - ETA: 5s - loss: 0.0918 - accuracy: 0.97 - ETA: 5s - loss: 0.0899 - accuracy: 0.97 - ETA: 5s - loss: 0.0964 - accuracy: 0.97 - ETA: 5s - loss: 0.0974 - accuracy: 0.97 - ETA: 5s - loss: 0.1007 - accuracy: 0.97 - ETA: 5s - loss: 0.0999 - accuracy: 0.97 - ETA: 4s - loss: 0.0992 - accuracy: 0.97 - ETA: 4s - loss: 0.0963 - accuracy: 0.97 - ETA: 4s - loss: 0.0967 - accuracy: 0.97 - ETA: 4s - loss: 0.0950 - accuracy: 0.97 - ETA: 4s - loss: 0.0920 - accuracy: 0.97 - ETA: 4s - loss: 0.0908 - accuracy: 0.97 - ETA: 4s - loss: 0.0893 - accu

12630/12630 [==============================] - ETA: 6s - loss: 0.0254 - accuracy: 1.00 - ETA: 5s - loss: 0.0398 - accuracy: 0.98 - ETA: 5s - loss: 0.0835 - accuracy: 0.97 - ETA: 5s - loss: 0.0771 - accuracy: 0.97 - ETA: 5s - loss: 0.0682 - accuracy: 0.97 - ETA: 5s - loss: 0.0621 - accuracy: 0.98 - ETA: 5s - loss: 0.0558 - accuracy: 0.98 - ETA: 5s - loss: 0.0586 - accuracy: 0.98 - ETA: 5s - loss: 0.0574 - accuracy: 0.98 - ETA: 5s - loss: 0.0654 - accuracy: 0.98 - ETA: 5s - loss: 0.0655 - accuracy: 0.98 - ETA: 5s - loss: 0.0729 - accuracy: 0.98 - ETA: 5s - loss: 0.0736 - accuracy: 0.97 - ETA: 5s - loss: 0.0743 - accuracy: 0.97 - ETA: 5s - loss: 0.0697 - accuracy: 0.97 - ETA: 5s - loss: 0.0724 - accuracy: 0.97 - ETA: 5s - loss: 0.0738 - accuracy: 0.97 - ETA: 4s - loss: 0.0734 - accuracy: 0.97 - ETA: 4s - loss: 0.0700 - accuracy: 0.97 - ETA: 4s - loss: 0.0682 - accuracy: 0.97 - ETA: 4s - loss: 0.0653 - accuracy: 0.97 - ETA: 4s - loss: 0.0688 - accuracy: 0.97 - ETA: 4s - loss: 0.0701 - accu

In [108]:
a=model.predict_classes(X_test)

#### Testing the model with test data set

In [113]:
accuracy_score(y_test,a)

0.9365499008592201

# Saving the model

In [123]:
model.save('newfolder')

INFO:tensorflow:Assets written to: newfolder\assets


# again loading the model to check wether model is saved properly or not

In [4]:
import tensorflow

In [5]:
model1 =tensorflow.keras.models.load_model('newfolder')

In [14]:
from sklearn.metrics import accuracy_score

In [12]:
predicted=model1.predict_classes(X_train)

predicted1=model1.predict_classes(X_test)

predicted2=model1.predict_classes(X_valid)

In [17]:
a0=accuracy_score(y_train,predicted)
a1=accuracy_score(y_test,predicted1)
a2=accuracy_score(y_valid,predicted2)

print(a0,a1,a2)

0.9995249406175772 0.9439351705508779 0.9306122448979591
